In [7]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

csv_dir = '/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/index_stats/'
def get_dataset_from_testcase(run):
    return run.split("_")[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/index_stats',
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge_alex/testrun',
]
index_stats = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    json_file = builtins.open(os.path.join(dir, test_case, 'input_results', 'inner'))
    run = (json.load(json_file))
    print(run)
    for stat in run['result']['index_stats']:
       stat['num_keys'] = run['result']['num_keys']
       stat['dataset'] = get_dataset_from_testcase(test_case)
       stat['epsilon'] = stat['index_name'][-3:]
       if stat['epsilon'] == '024':
        stat['epsilon'] = '1024'
       if stat['epsilon'] == '096':
        stat['epsilon'] = '4096'
       index_stats.append(stat)
df = pd.json_normalize(index_stats)
print(df.columns)
display(df)

{'command': 'sponge/index_stats/build/benchmark_runner sponge/index_stats/fb_threads=1/input_configs/inner', 'result': {'index_stats': [{'index_build_duration': 5095080, 'index_load_duration': 6395661690, 'index_name': 'btree256', 'index_size': 12627696}, {'index_build_duration': 6335686959, 'index_load_duration': 5267907911, 'index_name': 'flatpgm256', 'index_size': 4100112}, {'index_build_duration': 5934215332, 'index_load_duration': 4821454857, 'index_name': 'pgm256', 'index_size': 4143880}, {'index_build_duration': 3503137771, 'index_load_duration': 4772575093, 'index_name': 'radixspline1024', 'index_size': 955184512}, {'index_build_duration': 3549827970, 'index_load_duration': 4748310682, 'index_name': 'radixspline256', 'index_size': 958375200}, {'index_build_duration': 3433909174, 'index_load_duration': 4790488749, 'index_name': 'radixspline4096', 'index_size': 954524560}, {'index_build_duration': 57014222, 'index_load_duration': 3327732708, 'index_name': 'sampledflatpgm1024', 'i

,index_build_duration,index_load_duration,index_name,index_size,num_keys,dataset,epsilon
0,5095080,6395661690,btree256,12627696,199999999,fb,256
1,6335686959,5267907911,flatpgm256,4100112,199999999,fb,256
2,5934215332,4821454857,pgm256,4143880,199999999,fb,256
3,3503137771,4772575093,radixspline1024,955184512,199999999,fb,1024
4,3549827970,4748310682,radixspline256,958375200,199999999,fb,256
5,3433909174,4790488749,radixspline4096,954524560,199999999,fb,4096
6,57014222,3327732708,sampledflatpgm1024,731488,199999999,fb,1024
7,59407848,3320326909,sampledflatpgm256,3162320,199999999,fb,256
8,43650120,3296123830,sampledflatpgm4096,91008,199999999,fb,4096
9,63989704,3187768153,sampledpgm256,3189976,199999999,fb,256


In [10]:
rows = duckdb.sql(
        " SELECT index_name, dataset, index_build_duration, index_size FROM df "
        " ORDER BY index_build_duration"
    ).df()
fb_rmi = {'index_name': 'rmi', 'dataset': 'fb', 'index_build_duration': 22840080136, 'index_size': 402653200}
osm_rmi = {'index_name': 'rmi', 'dataset': 'osm', 'index_build_duration': 66079622803, 'index_size': 402653216}
books_rmi = {'index_name': 'rmi', 'dataset': 'books', 'index_build_duration': 74947767173, 'index_size': 402653216}
wiki_rmi = {'index_name': 'rmi', 'dataset': 'wiki', 'index_build_duration': 13247391814, 'index_size': 402653216}
rows.loc[len(rows)] = fb_rmi
rows.loc[len(rows)] = books_rmi
rows.loc[len(rows)] = osm_rmi
rows.loc[len(rows)] = wiki_rmi
rows['index_build_duration'] = rows['index_build_duration'] / 1e9
rows['index_size'] = rows['index_size'] /  1e6

def save_index_stats(dataset, path):
    print(dataset)
    tmp = duckdb.sql(f"SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='{dataset}' ORDER BY index_size").df()
    print(tmp.to_markdown(floatfmt='0.3f'))
    print(path)
    tmp.to_csv(os.path.join(csv_dir, f'{dataset}.txt'))


print(os.path.join(csv_dir, 'fb.txt'))
save_index_stats('fb', os.path.join(csv_dir, 'fb.txt'))

print("wiki")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='wiki' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

print("books")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='books' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

print("osm")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='osm' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/index_stats/fb.txt
fb


ValueError: Invalid format specifier